In [ ]:
import json
import os
import re
import textwrap

import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import silhouette_samples, silhouette_score

In [ ]:
# Define a function to pretty-print JSON with wrapped lines
def pretty_print_json(data, max_width=80):
    json_str = json.dumps(data, indent=4)
    wrapped_lines = []
    for line in json_str.splitlines():
        wrapped_lines.extend(textwrap.wrap(line, width=max_width))
    return "\n".join(wrapped_lines)

file_path = os.path.join("eval_baseline_emrqa", "eval_predictions.jsonl")
with open(file_path, "r") as file:
    predictions = [json.loads(line) for line in file]

# Filter failed examples where the predicted answer does not match the expected answer
failed_examples = [
    prediction for prediction in predictions
    if prediction['predicted_answer'] != prediction['answers']['text'][0] # for whatever reason this is a list
]

# Put Failed examples in a dataframe
failed = pd.DataFrame(failed_examples)
failed['answer'] = [v['text'][0] for v in failed["answers"].values]
# pd.set_option('display.max_colwidth', 20)
failed.columns
len(failed)

In [ ]:
############################
# Basic text utilities
############################

STOPWORDS = {
    "the", "a", "an", "of", "to", "and", "or", "in", "on", "for",
    "with", "is", "are", "was", "were", "be", "been", "being",
    "at", "by", "from", "as", "that", "this", "these", "those",
    "it", "its", "he", "she", "they", "them", "his", "her", "their",
    "we", "you", "i", "but", "if", "then", "so", "because", "when",
    "while", "about", "into", "out", "up", "down", "over", "under"
}

WORD_RE = re.compile(r"\w+")
DATE_RE = re.compile(
    r"\b(?:\d{1,2}[/-]\d{1,2}[/-]\d{2,4}|\d{4}|jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|"
    r"may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:t(?:ember)?)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)\b",
    re.IGNORECASE,
)
NUMBER_RE = re.compile(r"\b\d+(?:\.\d+)?\b")

def normalize(text: str) -> str:
    return " ".join(text.lower().split())

def tokenize(text: str):
    return WORD_RE.findall(normalize(text))

def jaccard_similarity(a_tokens, b_tokens):
    a_set, b_set = set(a_tokens), set(b_tokens)
    if not a_set or not b_set:
        return 0.0
    inter = len(a_set & b_set)
    union = len(a_set | b_set)
    return inter / union

# to identify repetitive/uninformative answers
def shannon_entropy(tokens):
    if not tokens:
        return 0.0
    counts = Counter(tokens)
    total = sum(counts.values())
    ent = 0.0
    for c in counts.values():
        p = c / total
        ent -= p * math.log2(p)
    return ent

def count_dates(text: str) -> int:
    return len(DATE_RE.findall(text))

def count_numbers(text: str) -> int:
    return len(NUMBER_RE.findall(text))

def stopword_ratio(tokens):
    if not tokens:
        return 1.0
    sw = sum(1 for t in tokens if t in STOPWORDS)
    return sw / len(tokens)


############################
# Dataset-level stats
############################

def compute_dataset_stats(examples):
    """
    Compute basic stats over gold spans & question-span similarity
    so rules can use dynamic thresholds.
    """
    gold_lengths = []
    qg_sims = []

    for ex in examples:
        gold = ex["answers"]["text"][0]
        question = ex["question"]
        gold_tokens = tokenize(gold)
        q_tokens = tokenize(question)

        if not gold_tokens:
            continue

        gold_lengths.append(len(gold_tokens))
        qg_sims.append(jaccard_similarity(q_tokens, gold_tokens))

    if not gold_lengths:
        # degenerate case
        return {
            "median_gold_len": 0,
            "mad_gold_len": 0,
            "p95_gold_len": 0,
            "mean_qg_sim": 0.0,
            "std_qg_sim": 0.0,
        }

    # median and MAD
    med_len = median(gold_lengths)
    abs_dev = [abs(l - med_len) for l in gold_lengths]
    mad_len = median(abs_dev)

    # rough p95
    sorted_lens = sorted(gold_lengths)
    idx_95 = int(0.95 * (len(sorted_lens) - 1))
    p95_len = sorted_lens[idx_95]

    # mean and std of q–gold similarity
    if qg_sims:
        mean_sim = sum(qg_sims) / len(qg_sims)
        var_sim = sum((s - mean_sim) ** 2 for s in qg_sims) / max(len(qg_sims) - 1, 1)
        std_sim = math.sqrt(var_sim)
    else:
        mean_sim, std_sim = 0.0, 0.0

    return {
        "median_gold_len": med_len,
        "mad_gold_len": mad_len,
        "p95_gold_len": p95_len,
        "mean_qg_sim": mean_sim,
        "std_qg_sim": std_sim,
    }


############################
# Rule implementations
############################

# Extremely long answers may indicate annotation errors where too much text was selected
def rule1_length_anomaly(gold_tokens, stats):
    L = len(gold_tokens)
    med = stats["median_gold_len"]
    mad = stats["mad_gold_len"]
    p95 = stats["p95_gold_len"]

    too_long_vs_p95 = L > p95
    too_long_vs_mad = mad > 0 and L > med + 2 * mad
    return too_long_vs_p95 or too_long_vs_mad

def split_clauses(text: str):
    # very rough clause segmentation
    clauses = re.split(r'[,\n;]| and | or ', text)
    # keep clauses that have at least 2 non-stopword tokens
    clean_clauses = []
    for c in clauses:
        toks = tokenize(c)
        non_sw = [t for t in toks if t not in STOPWORDS]
        if len(non_sw) >= 2:
            clean_clauses.append(c.strip())
    return clean_clauses

# Multi-clause answers may be over segmented or contain irrelevant information
def rule2_multi_clause(gold_text: str):
    clauses = split_clauses(gold_text)
    return len(clauses) > 1


# def rule3_low_question_similarity(question_tokens, gold_tokens, stats, k=1.0):
#     sim = jaccard_similarity(question_tokens, gold_tokens)
#     mean_sim = stats["mean_qg_sim"]
#     std_sim = stats["std_qg_sim"]
#     # flag if sim is much lower than typical q–gold sim
#     threshold = mean_sim - k * std_sim
#     # if std_sim == 0, threshold == mean_sim; low sim will be 0, so flagged
#     return sim < threshold

# Most commonly observed issue: The Model extracts a more relevant portion of an over-annotated gold answer
def rule4_pred_inside_gold_and_better_align(pred_text, gold_text, question_tokens):
    gold_norm = normalize(gold_text)
    pred_norm = normalize(pred_text)
    # substring check in normalized space
    if pred_norm and pred_norm in gold_norm:
        gold_tokens = tokenize(gold_text)
        pred_tokens = tokenize(pred_text)
        sim_gold = jaccard_similarity(question_tokens, gold_tokens)
        sim_pred = jaccard_similarity(question_tokens, pred_tokens)
        return sim_pred > sim_gold + 0.1  # margin; tune if needed
    return False

def classify_question_type(question: str):
    q = question.lower()
    if any(x in q for x in ["when", "date", "time", "year", "day", "month"]):
        return "time"
    if "how many" in q or "number of" in q or "count" in q:
        return "number"
    # you can expand this as needed (medication, lab, diagnosis, etc.)
    return "other"

def rule5_question_type_mismatch(gold_text: str, question: str):
    q_type = classify_question_type(question)
    if q_type == "time":
        n_dates = count_dates(gold_text)
        return n_dates != 1  # we expect exactly one
    if q_type == "number":
        n_nums = count_numbers(gold_text)
        return n_nums != 1
    return False  # for "other" we don't use this rule

def rule6_multiple_occurrences(gold_text: str, context: str):
    gold_norm = normalize(gold_text)
    ctx_norm = normalize(context)
    if not gold_norm:
        return False
    return ctx_norm.count(gold_norm) > 1

# lot of answers only have '.' or ',' so it must be predicting wrong span.
def rule7_boundary_weirdness(gold_text: str):
    stripped = gold_text.strip()
    if not stripped:
        return False
    starts_bad = stripped[0] in {",", ";", ":", "-", "."}
    ends_bad = stripped[-1] in {",", ";", ":", "-", "(", "/"}
    return starts_bad or ends_bad

def rule8_low_entropy(gold_tokens):
    ent = shannon_entropy(gold_tokens)
    # heuristic: very low entropy for longer spans is suspicious
    return len(gold_tokens) >= 5 and ent < 1.0

# def rule9_high_stopword_ratio(gold_tokens, threshold=0.7):
#     return stopword_ratio(gold_tokens) > threshold

# # prediction better than answers
# def rule10_pred_better_answers_question(pred_text, gold_text, question_tokens):
#     gold_tokens = tokenize(gold_text)
#     pred_tokens = tokenize(pred_text)
#     sim_gold = jaccard_similarity(question_tokens, gold_tokens)
#     sim_pred = jaccard_similarity(question_tokens, pred_tokens)
#     return sim_pred > sim_gold + 0.1  # margin


############################
# Overall dataset-error detector
############################

def detect_dataset_error(example, stats, min_rules_triggered=2):
    """
    Returns (is_dataset_error, details_dict)
    details_dict tells you which rules fired.
    """
    context = example["context"]
    question = example["question"]
    gold_text = example["answers"]["text"][0]
    pred_text = example.get("predicted_answer", "")

    gold_tokens = tokenize(gold_text)
    question_tokens = tokenize(question)

    # individual rule flags
    flags = {}

    flags["rule1_length_anomaly"] = rule1_length_anomaly(gold_tokens, stats)
    flags["rule2_multi_clause"] = rule2_multi_clause(gold_text)
    #flags["rule3_low_q_sim"] = rule3_low_question_similarity(question_tokens, gold_tokens, stats)
    flags["rule4_pred_inside_gold_better_align"] = rule4_pred_inside_gold_and_better_align(
        pred_text, gold_text, question_tokens
    )
    flags["rule5_qtype_mismatch"] = rule5_question_type_mismatch(gold_text, question)
    flags["rule6_multi_occurrences"] = rule6_multiple_occurrences(gold_text, context)
    flags["rule7_boundary_weirdness"] = rule7_boundary_weirdness(gold_text)
    flags["rule8_low_entropy"] = rule8_low_entropy(gold_tokens)
    # flags["rule9_high_stopword_ratio"] = rule9_high_stopword_ratio(gold_tokens)
    # flags["rule10_pred_better_answer"] = rule10_pred_better_answers_question(pred_text, gold_text, question_tokens)

    num_true = sum(flags.values())
    is_error = num_true >= min_rules_triggered

    return is_error, flags


stats = compute_dataset_stats(predictions)

IndentationError: expected an indented block after function definition on line 217 (3037764688.py, line 233)

In [7]:

results = []
for ex in predictions:
    is_err, flags = detect_dataset_error(ex, stats, min_rules_triggered=2)
    results.append({
        "id": ex["id"],
        "is_dataset_error": is_err,
        "flags": flags,
        "gold": ex["answers"]["text"][0],
        "pred": ex.get("predicted_answer", ""),
        "question": ex["question"],
    })

import pandas as pd

df = pd.DataFrame(results)

total = len(df)
num_ds_errors = df["is_dataset_error"].sum()

print("Total examples:", total)
print("Dataset errors:", num_ds_errors)
print("Percentage:", num_ds_errors / total * 100, "%")

def is_failure(row):
    return row["pred"] != row["gold"]

df["is_failure"] = df.apply(is_failure, axis=1)

failed_df = df[df["is_failure"] == True]

fail_total = len(failed_df)
fail_ds_errors = failed_df["is_dataset_error"].sum()

print("Model failures:", fail_total)
print("Dataset errors among failures:", fail_ds_errors)
print("Percentage:", fail_ds_errors / fail_total * 100, "%")
rule_columns = list(results[0]["flags"].keys())

rule_counts = {rule: 0 for rule in rule_columns}

for r in results:
    for rule, val in r["flags"].items():
        if val:
            rule_counts[rule] += 1

print("=== Rule Trigger Rates ===")
for rule, count in rule_counts.items():
    print(f"{rule}: {count} ({count/total*100:.2f}%)")


Total examples: 32739
Dataset errors: 11249
Percentage: 34.35963224289074 %
Model failures: 3248
Dataset errors among failures: 2829
Percentage: 87.09975369458128 %
=== Rule Trigger Rates ===
rule1_length_anomaly: 4176 (12.76%)
rule2_multi_clause: 10250 (31.31%)
rule3_low_q_sim: 3002 (9.17%)
rule4_pred_inside_gold_better_align: 19 (0.06%)
rule5_qtype_mismatch: 722 (2.21%)
rule6_multi_occurrences: 1871 (5.71%)
rule7_boundary_weirdness: 21825 (66.66%)
rule8_low_entropy: 0 (0.00%)
rule9_high_stopword_ratio: 0 (0.00%)
rule10_pred_better_answer: 57 (0.17%)


In [4]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_colwidth', None)
# failed[failed['question'].str.lower().str.startswith('has')].head(50)
gemfibrozil_examples = failed[failed['question'].str.lower().str.contains('gemfibrozil')]
gemfibrozil_examples.head(5)

10

In [ ]:
len(failed[failed['predicted_answer'] == ',']), len(failed[failed['predicted_answer'] == '.'])

###  Baseline EMRQA prediction error analysis
- High-level error patterns for `eval_baseline_emrqa/eval_predictions.jsonl`
- Label wrong predictions as `truncated_span`, `overlong_span`, `partial_overlap`, or `no_overlap` and show counts/examples.

In [ ]:
# Helpers
token_re = re.compile(r"\w+")
relation = ["truncated_span", "overrun_span", "partial_overlap", "no_overlap"]

def norm(text):
    return " ".join(text.lower().split())

def tokens(text: str):
    return token_re.findall(text.lower())

def span_relation(prediction: str, actual: str):
    p = norm(prediction)
    a = norm(actual)
    if p and p in a:
        return relation[0]
    if a and a in p:
        return relation[1]
    if set(tokens(p)) & set(tokens(a)):
        return relation[2]
    return relation[3]

# Add a new column to the failed dataframe for the span relation
failed['relation'] = failed.apply(
    lambda row: span_relation(
        row.get("predicted_answer", ""),
        row.get("answer", "")
    ),
    axis=1
)

# Display counts from the failed dataframe
print(f"Total wrong predictions: {len(failed)}")
print("Wrong predictions by type:")
print(failed["relation"].value_counts())

# Use the following filter to deep dive
# Sample examples from each relation type
samples = []
for rel in relation:
    rel_samples = failed[failed["relation"] == rel][["id", "question", "predicted_answer", "answer", "relation"]].head(3)
    samples.append(rel_samples)

pd.concat(samples, ignore_index=True)

### Deep Dive into Baseline EMRQA Prediction Errors

In [ ]:
failed['gold_norm'] = failed['answer'].apply(norm)
failed['pred_norm'] = failed['predicted_answer'].apply(norm)
failed["clust_text"] = failed.apply(
    lambda r: f"Q: {r['question']} GOLD: {r['gold_norm']} PRED: {r['pred_norm']}",
    axis=1
)

Tfidf + k-means did not yield meaningful clusters here.

In [ ]:
vectorizer = TfidfVectorizer(
    max_features=3000,
    ngram_range=(1, 2),
    min_df=5
)

X = vectorizer.fit_transform(failed["clust_text"])
X.shape

In [ ]:
k = 10
kmeans = KMeans(n_clusters=k, random_state=0, n_init=10, max_iter=100)
cluster_labels = kmeans.fit_predict(X)

failed["cluster"] = cluster_labels
cluster_sizes = failed["cluster"].value_counts().sort_index()
cluster_sizes

Per Cluster Analysis

In [ ]:
cluster_summaries = {}

for c in sorted(failed["cluster"].unique()):
    sub = failed[failed["cluster"] == c]

    qs = sub["question"].astype(str).tolist()
    if len(qs) < 5:
        continue

    cv = CountVectorizer(
        max_features=2000,
        ngram_range=(1,3),
        stop_words="english"
    )
    Xq = cv.fit_transform(qs)

    sums = np.asarray(Xq.sum(axis=0)).ravel()
    terms = np.array(cv.get_feature_names_out())
    top_idx = sums.argsort()[::-1][:15]

    cluster_summaries[c] = {
        "n": len(sub),
        "top_terms": list(terms[top_idx]),
        "error_dist": sub["relation"].value_counts().to_dict()
    }

cluster_summaries

In [ ]:
for c in sorted(cluster_summaries.keys()):
    info = cluster_summaries[c]
    print("="*70)
    print(f"Cluster {c}  |  n = {info['n']}")
    print("Top terms:", ", ".join(info["top_terms"]))
    print("Error types:", info["error_dist"])
    print()

In [ ]:
ks = list(range(2, 25))
sse = []

for k in ks:
    km = KMeans(n_clusters=k, random_state=0, n_init=10, max_iter=200)
    km.fit(X)
    sse.append(km.inertia_)

plt.plot(ks, sse, marker='o')
plt.xlabel("k")
plt.ylabel("Inertia (SSE)")
plt.title("Elbow Method")
plt.show()

Cluster error types in failed examples for further analysis. This uses sentence transformers to embed the question, context, and predicted/true answers, then clusters them using KMeans. This seems to work better.

In [ ]:
model = SentenceTransformer("all-mpnet-base-v2")


embed_texts = failed["clust_text"].tolist()
embeddings = model.encode(
    embed_texts,
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True
)

embeddings.shape

In [ ]:
pca = PCA(n_components=2, random_state=42)
pca_2d = pca.fit_transform(embeddings)

failed["pca_x"] = pca_2d[:, 0]
failed["pca_y"] = pca_2d[:, 1]

In [ ]:
ks = list(range(2, 16))
sil_scores = []

for k in ks:
    km = KMeans(
        n_clusters=k,
        random_state=0,
        n_init=10,
        max_iter=300
    )
    labels = km.fit_predict(embeddings)
    score = silhouette_score(embeddings, labels)
    sil_scores.append(score)
    print(f"k={k:2d}  silhouette={score:.4f}")

plt.figure(figsize=(6,4))
plt.plot(ks, sil_scores, marker="o")
plt.xlabel("k")
plt.ylabel("Silhouette score")
plt.title("Silhouette scores (KMeans on embeddings)")
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
best_k = 8

kmeans = KMeans(
    n_clusters=best_k,
    random_state=0,
    n_init=10,
    max_iter=300
)
k_labels = kmeans.fit_predict(embeddings)

failed["cluster_kmeans"] = k_labels
failed["cluster_kmeans"].value_counts().sort_index()

In [ ]:
plt.figure(figsize=(8, 6))

for cid in sorted(failed["cluster_kmeans"].unique()):
    mask = failed["cluster_kmeans"] == cid
    plt.scatter(
        failed.loc[mask, "pca_x"],
        failed.loc[mask, "pca_y"],
        s=5,
        alpha=0.7,
        label=f"cluster {cid}"
    )

plt.xlabel("PCA-1")
plt.ylabel("PCA-2")
plt.title(f"KMeans (k={best_k}) on embeddings (PCA 2D)")
plt.legend(markerscale=3, bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()

In [ ]:
cluster_k_info = {}

for cid in sorted(failed["cluster_kmeans"].unique()):
    sub = failed[failed["cluster_kmeans"] == cid]
    n = len(sub)
    rel_dist = sub["relation"].value_counts().to_dict()
    
    top_terms = []
    if n >= 5:
        cv = CountVectorizer(
            max_features=2000,
            ngram_range=(1,2),
            stop_words="english"
        )
        Xq = cv.fit_transform(sub["question"].astype(str).tolist())
        sums = np.asarray(Xq.sum(axis=0)).ravel()
        terms = np.array(cv.get_feature_names_out())
        top_idx = sums.argsort()[::-1][:15]
        top_terms = list(terms[top_idx])
    
    cluster_k_info[cid] = {
        "n": n,
        "relation_dist": rel_dist,
        "top_terms": top_terms,
    }

for cid, info in cluster_k_info.items():
    print("="*80)
    print(f"Cluster {cid}  |  n = {info['n']}")
    print("Error-type distribution:", info["relation_dist"])
    print("Top question terms:", ", ".join(info["top_terms"]))
    print()

### Manual Review of Sampled Errors

- No. of questions in the train dataset that start with "has" is 62872/130,956
- No. of questions in the validation dataset that start with "has" is 15913/32739
- No. of questions in the failed examples that start with "has" is 1423/3248

This examines the whole dataset for train and eval to see how many questions start with "has", 'why' etc.

In [ ]:
train = load_dataset("Eladio/emrqa-msquad", split="train")

# Filter questions that start with "why"
why_questions = train.filter(lambda example: example["question"].lower().startswith("why"))

why_questions_df = pd.DataFrame(why_questions)
len(why_questions_df)

In [ ]:
eval = load_dataset("Eladio/emrqa-msquad", split="validation")

# Filter questions that start with "has"
has_questions = eval.filter(lambda example: example["question"].lower().startswith("has"))

# Display the filtered examples
has_questions_df = pd.DataFrame(has_questions)
has_questions_df['question']

Insulin

In [ ]:
insulin_questions = failed[failed['question'].str.contains('insulin', case=False, na=False)]
len(insulin_questions)

Chest pain

In [ ]:
chest_pain_questions = failed[failed['question'].str.contains('chest pain', case=False, na=False)]
len(chest_pain_questions)

Medication

In [ ]:
medication_questions = failed[failed['question'].str.contains('medication', case=False, na=False)]
len(chest_pain_questions)

In [ ]:
# Authors: The scikit-learn developers
# SPDX-License-Identifier: BSD-3-Clause

# Generating the sample data from make_blobs
# This particular setting has one distinct cluster and 3 clusters placed close
# together.
X, y = make_blobs(
    n_samples=500,
    n_features=2,
    centers=4,
    cluster_std=1,
    center_box=(-10.0, 10.0),
    shuffle=True,
    random_state=1,
)  # For reproducibility

range_n_clusters = [2, 3, 4, 5, 6]

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg,
    )

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            ith_cluster_silhouette_values,
            facecolor=color,
            edgecolor=color,
            alpha=0.7,
        )

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(
        X[:, 0], X[:, 1], marker=".", s=30, lw=0, alpha=0.7, c=colors, edgecolor="k"
    )

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(
        centers[:, 0],
        centers[:, 1],
        marker="o",
        c="white",
        alpha=1,
        s=200,
        edgecolor="k",
    )

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker="$%d$" % i, alpha=1, s=50, edgecolor="k")

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(
        "Silhouette analysis for KMeans clustering on sample data with n_clusters = %d"
        % n_clusters,
        fontsize=14,
        fontweight="bold",
    )

plt.show()